Get NDA data dictionary items pertaining to NIH Toolbox data for mapping visibility and QC

In [102]:
#read crosswalkfile
pathin="/home/petra/UbWinSharedSpace1/ccf-nda-behavioral/PycharmToolbox/Ipad2NDA_withCrosswalk/NIHToolbox2NDA/"
crosswalkfile="NIH_Toolbox_crosswalk_HCP.csv"
crosswalk=pd.read_csv(crosswalkfile,header=0,low_memory=False, encoding = "ISO-8859-1")
crosswalk.head()


,Inst,template,inst_short,Source,nda_structure,nda_element,hcp_variable,action_requested,hcp_variable_upload,specialty_code,requested_python
0,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,version_form,Assessment_Name,NaN,Assessment_Name,NaN,NaN
1,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,nih_tlbx_fctsc,Fully_Corrected_T_score,NaN,Fully_Corrected_T_score,NaN,NaN
2,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,primary_language,Language,Please rename 'Language' to 'primary_language'...,primary_language,NaN,studydata['primary_language']=studydata['Langu...
3,Cognition Composite Scores,cogcomp01_template,cogcomp01,HCPD HCPA,cogcomp01,version_form,Assessment_Name,NaN,Assessment_Name,1,NaN
4,Cognition Composite Scores,cogcomp01_template,cogcomp01,HCPD HCPA,cogcomp01,interview_language,Language,NaN,Language,1,NaN


In [63]:
# Programatically retreive data from the dictionary service for every hcp_variable that has 
# been mapped to the NDA by their data curators for line item comparisons between Toolbox Data Dicitonary and 
# NDA data dictionary

import requests
import json
import pandas as pd

#example to expand
shortname = 'cogcomp01'
varlist=['subjectkey','src_subject_id']
r = requests.get('https://ndar.nih.gov/api/datadictionary/datastructure/{}'
                 .format(shortname),
                  headers={'Accept':'application/json'})
structure = json.loads(r.text)
#structure.keys()
df=pd.DataFrame(structure['dataElements'])
df=df[['name','description','valueRange','notes']]
dfxwalk=df.loc[df.name.isin(varlist)] 
dfxwalk


,name,description,valueRange,notes
0,subjectkey,The NDAR Global Unique Identifier (GUID) for r...,NDAR*,None
1,src_subject_id,Subject ID how it's defined in lab/project,None,None


In [88]:
#for a given structure (shortname), grab all the metadata for a list of elements as a dataframe
def getNDAdetails(structure_name=shortname,crosswalk=crosswalk):
    varlist=crosswalk.loc[crosswalk.nda_structure==structure_name,'nda_element'].to_list()
    r = requests.get('https://ndar.nih.gov/api/datadictionary/datastructure/{}'
                 .format(structure_name),
                  headers={'Accept':'application/json'})
    structure = json.loads(r.text)
    df=pd.DataFrame(structure['dataElements'])
    df2=df[['name','description','valueRange','notes']].copy()
    dfxwalk=df2.loc[df2.name.isin(varlist)].copy() 
    dfxwalk['nda_structure']=structure_name
    dfxwalk=dfxwalk.rename(columns={'name':'nda_element'})
    return dfxwalk



In [93]:
#test
#dfxwalk=getNDAdetails(structure_name='prang01')
#dfxwalk
#dfxwalk=getNDAdetails(structure_name='tlbx_wellbeing01')
#dfxwalk
dfxwalk=getNDAdetails(structure_name='tlbx_motor01')
dfxwalk

,nda_element,description,valueRange,notes,nda_structure
5,version_form,Form used/assessment name,None,None,tlbx_motor01
8,en6m50_2m_10ft_marks,Endurance - 2 minute walk - Number of 10 ft marks,None,None,tlbx_motor01
9,en6m50_2m_ft,Endurance - 2 minute walk - Feet,None,None,tlbx_motor01
10,en6m50_2m_inches,Endurance - 2 minute walk - Inches,None,None,tlbx_motor01
11,en6m50_2m_turns,Endurance - 2 minute walk - Number of course t...,None,None,tlbx_motor01
13,end_2m_standardsc,Endurance -Normative Standard Score,None,None,tlbx_motor01
19,grip_standardsc_dom,9-Hole Grip Strength Dominant Hand Normative S...,None,None,tlbx_motor01
20,grip_standardsc_nondom,9-Hole Grip Strength Non-Dominant Hand Normati...,None,None,tlbx_motor01
25,lo4mt1,Locomotion - 4 meter walk - Trial 1 (Usual Pac...,None,None,tlbx_motor01
26,lo4mt2,Locomotion - 4 meter walk - Trial 2 (Fast Pace...,None,None,tlbx_motor01


In [98]:
#append all this information together for a merge
allmetadata=pd.DataFrame()
for structshort in crosswalk.nda_structure.unique():
    try:
        dfxwalk=getNDAdetails(structure_name=structshort)
        allmetadata=pd.concat([allmetadata,dfxwalk])
    except:
        print('could not process {}'.format(structshort))
 



In [99]:
allmetadata

,nda_element,description,valueRange,notes,nda_structure
5,primary_language,Subject's Primary Language,None,None,tlbx_fearanx01
7,version_form,Form used/assessment name,None,None,tlbx_fearanx01
8,pedrepanx50,In the past 7 days I worried when I was at home,1::5,1=Never; 2=Almost Never; 3=Sometimes; 4=Often;...,tlbx_fearanx01
9,pedrepanx51,In the past 7 days I got scared really easily,1::5,1=Never; 2=Almost Never; 3=Sometimes; 4=Often;...,tlbx_fearanx01
10,pedproxanx29,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01
11,pedproxanx31,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01
12,pedproxanx32,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01
13,pedproxanx35,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01
14,pedproxanx36,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01
15,pedproxanx37,Please indicate how often or true the behavior...,0::2,0=Never or not true; 1=Sometimes or somewhat t...,tlbx_fearanx01


In [100]:
#now merge this data into the crosswalk and write the new csv
newcrosswalk=pd.merge(crosswalk,allmetadata,how='left',on=['nda_structure','nda_element'])

In [101]:
newcrosswalk.head()

,Inst,template,inst_short,Source,nda_structure,nda_element,hcp_variable,action_requested,hcp_variable_upload,specialty_code,requested_python,description,valueRange,notes
0,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,version_form,Assessment_Name,NaN,Assessment_Name,NaN,NaN,Form used/assessment name,None,None
1,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,nih_tlbx_fctsc,Fully_Corrected_T_score,NaN,Fully_Corrected_T_score,NaN,NaN,Fully-Corrected T-Score,0::120,None
2,Anxiety Summary Parent Report (3-7),Anxiety_Summary_3-7.tlbx_fearanx01_template,Anxiety_Summary_3-7,HCPD,tlbx_fearanx01,primary_language,Language,Please rename 'Language' to 'primary_language'...,primary_language,NaN,studydata['primary_language']=studydata['Langu...,Subject's Primary Language,None,None
3,Cognition Composite Scores,cogcomp01_template,cogcomp01,HCPD HCPA,cogcomp01,version_form,Assessment_Name,NaN,Assessment_Name,1,NaN,Form used/assessment name,None,None
4,Cognition Composite Scores,cogcomp01_template,cogcomp01,HCPD HCPA,cogcomp01,interview_language,Language,NaN,Language,1,NaN,Language Used in the Interview,None,None


In [107]:
newcrosswalk.to_csv(pathin+crosswalkfile,index=False)#crosswalkfile,index=False)